Web Scraping: extraer datos del contenido HTML de un sitio web

Realizar ese proceso contenido no autorizado por el autor es ilegal!!

<a href="https://es.wikipedia.org/wiki/Web_scraping" target="_blank">Web scraping</a></br>
<a href="https://es.wikipedia.org/wiki/Est%C3%A1ndar_de_exclusi%C3%B3n_de_robots" target="_blank">Estándar de exclusión de robots</a></br>
<a href="https://www.imperva.com/learn/application-security/web-scraping-attack/" target="_blank">What is web scraping</a></br>
<a href="https://www.octoparse.com/blog/5-things-you-need-to-know-before-scraping-data-from-facebook" target="_blank">5 Things You Need to Know Before Scraping Data From Facebook</a></br>
<a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/" target="_blank">Beautiful Soup Documentation</a></br>

In [1]:
#https://archive.ics.uci.edu/ml/datasets/auto+mpg
#Auto MPG Data Set
#Inciar el servidor web localmente: 
#python -m http.server
import requests
respuesta = requests.get("http://localhost:8000/auto_mpg.html")

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /auto_mpg.html (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001A8080EA520>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [3]:
#Importar la clase BeautifulSoup del m
from bs4 import BeautifulSoup
pagina = BeautifulSoup(respuesta.text, "html.parser" )
#pagina?
#print(pagina.prettify())
#Demo. Chrome Developer Tools: Explorar la estructura de la página para identificar los elementos que contienen los datos


In [10]:
#print(pagina.body)
#print(pagina.body.find(name="div", attrs={"id":"car-1"}())
#print(len(pagina.body.find_all(name="div", attrs={"class":"car_block"})))
#print(pagina.body.find_all(name="div", attrs={"class":"car_block"})[0])
car_divs = pagina.body.find_all(name="div", attrs={"class":"car_block"})
div = car_divs[0]
#print(div.prettify())
#print(div)
#print(div.text) #Sin elementos HTML  :)
#print(div.stripped_strings) #generador, similar a un iterable
#print(list(div.stripped_strings))

<div class="car_block" id="car-1">
 <span class="car_name">
  Chevrolet Chevelle Malibu
 </span>
 <span class="from">
  (1970, USA)
 </span>
 <br/>
 Achieves
 <span class="mpg">
  18.0 mpg
 </span>
 with
 <span class="cylinders">
  8
 </span>
 cylinders backed by
 <span class="horsepower">
  130
 </span>
 hp, 307.0 cubic inches of displacement, weighing
 <span class="weight">
  3,504
 </span>
 lbs with 0-60 mph acceleration in
 <span class="acceleration">
  12.0
 </span>
 seconds
</div>



In [69]:
#div.find_all("span")
div.find("span", attrs={"class":"mpg"}).text
div.find("span", attrs={"class":"mpg"}).text.split(" ")[0]

'18.0'

In [11]:
print(div.prettify())
import re
re.findall(r'.* (\d+.\d+) cubic inches', div.text)[0]
#Continuar experimentando estrategias para extraer datos
#Finalmente, crear funciones para cada item de datos y procesar los elementos que contienen loa datos

<div class="car_block" id="car-1">
 <span class="car_name">
  Chevrolet Chevelle Malibu
 </span>
 <span class="from">
  (1970, USA)
 </span>
 <br/>
 Achieves
 <span class="mpg">
  18.0 mpg
 </span>
 with
 <span class="cylinders">
  8
 </span>
 cylinders backed by
 <span class="horsepower">
  130
 </span>
 hp, 307.0 cubic inches of displacement, weighing
 <span class="weight">
  3,504
 </span>
 lbs with 0-60 mph acceleration in
 <span class="acceleration">
  12.0
 </span>
 seconds
</div>



'307.0'

In [4]:
#Proceso extraccion
import csv
import re
import requests
from bs4 import BeautifulSoup

def extraer_territorio_anio(div_automovil):
    str_from = div_automovil.find("span", attrs={"class":"from"}).text
    anio, territorio = str_from.strip('()').split(',')
    anio = int(anio.strip())
    territorio = territorio.strip()
    return territorio, anio

def extraer_mpg(div_automovil):
    mpg_str = div_automovil.find("span", attrs={"class":"mpg"}).text
    try:
        mpg = float(mpg_str.split(' ')[0])
    except ValueError:
        mpg = "NULL"
    return mpg

def extraer_caballos_potencia(div_automovil):
    caballos_potencia_str = div_automovil.find('span', class_='horsepower').text
    try:
        caballos_potencia = float(caballos_potencia_str)
    except ValueError:
        caballos_potencia = "NULL"
    return caballos_potencia

def extraer_desplazamiento(div_automovil_text):
    str_desplazamiento = re.findall(r'.* (\d+.\d+) cubic inches', div_automovil_text)[0]
    desplazamiento = float(str_desplazamiento)
    return desplazamiento  

def extraer_datos(div_automovil):
    nombre = div_automovil.find('span', class_='car_name').text
    str_cilindros = div_automovil.find('span', class_='cylinders').text
    cilindros = int(str_cilindros)
    str_peso = div_automovil.find('span', class_='weight').text
    peso = int(str_peso.replace(',', ''))
    territorio, anio = extraer_territorio_anio(div_automovil)
    aceleracion = float(div_automovil.find('span', class_='acceleration').text)
    mpg = extraer_mpg(div_automovil)
    caballos_potencia = extraer_caballos_potencia(div_automovil)
    desplazamiento = extraer_desplazamiento(div_automovil.text)
    fila = dict(nombre=nombre,
               cilindros=cilindros,
               peso=peso,
               anio=anio,
               territorio=territorio,
               aceleracion=aceleracion,
               mpg=mpg,
               caballos_potencia=caballos_potencia,
               desplazamiento=desplazamiento)
    return fila

def extraer_datos_automoviles(pagina):
    """Extract information from repeated divisions"""
    divs_automoviles = pagina.body.find_all(name="div", attrs={"class":"car_block"})
    filas = []
    for div_automovil in divs_automoviles:
        fila = extraer_datos(div_automovil)
        filas.append(fila)
    print(f"Se extraido {len(filas)} filas")
    print(filas[0])
    print(filas[-1])

    with open("datos_automoviles.csv", "w") as f:
        writer = csv.DictWriter(f, fieldnames=fila.keys())
        writer.writeheader()
        writer.writerows(filas)
        
respuesta = requests.get("http://localhost:8000/auto_mpg.html")
pagina = BeautifulSoup(respuesta.text, "html.parser" )
extraer_datos_automoviles(pagina)

Se extraido 406 filas
{'nombre': 'Chevrolet Chevelle Malibu', 'cilindros': 8, 'peso': 3504, 'anio': 1970, 'territorio': 'USA', 'aceleracion': 12.0, 'mpg': 18.0, 'caballos_potencia': 130.0, 'desplazamiento': 307.0}
{'nombre': 'Chevy S-10', 'cilindros': 4, 'peso': 2720, 'anio': 1982, 'territorio': 'USA', 'aceleracion': 19.4, 'mpg': 31.0, 'caballos_potencia': 82.0, 'desplazamiento': 119.0}
